### Yield Curve Model Optimization work

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fredapi as fa

import statsmodels.api as sm

# ignore warnings if not working on changing code
import warnings
warnings.filterwarnings('ignore')

In [51]:
### use this fact if french fama
# facts = pd.read_csv("Factors.csv",header=3)
# facts = facts.drop("String",axis=1)
# facts['Date'] = pd.to_datetime(facts['Date'])
# facts = facts.set_index("Date")
# facts = facts.apply(pd.to_numeric)/100

###use these facts if you want 
facts = pd.read_excel("s_p_factor_indices.xlsx",header=1).dropna()
facts['Date'] = pd.to_datetime(facts['Date']) + pd.DateOffset(1)
facts = facts.set_index("Date")

#for getting the date label added
yield_mod =pd.read_excel('HY_label.xls',usecols=['Date','Label'])
yield_mod['Date'] = pd.to_datetime(yield_mod['Date'])
yield_mod = yield_mod.set_index("Date")

##add this if label index 0-3
#yield_mod['Label'] = yield_mod['Label'] + 1


yc_returns = pd.concat([facts,yield_mod],axis=1).dropna()
yc_returns

,min_var,revenue,growth,value,quality,Momentum,SPX,Label
Date,,,,,,,,
1999-04-01,0.013656,0.027457,0.048271,-0.003931,0.014442,0.067860,0.040005,1.0
1999-05-01,0.084371,0.075030,0.027143,0.120793,0.018472,-0.002146,0.038729,1.0
1999-06-01,0.025541,-0.016499,-0.012791,0.009738,-0.019100,-0.035856,-0.023617,1.0
1999-07-01,0.028780,0.037311,0.093956,0.018867,0.044987,0.087432,0.055497,1.0
1999-09-01,-0.017970,-0.015988,0.008653,-0.005377,0.010913,0.010087,-0.004945,4.0
...,...,...,...,...,...,...,...,...
2022-02-01,-0.050134,-0.017723,-0.133407,0.020013,-0.040978,-0.063998,-0.051747,3.0
2022-03-01,-0.029301,-0.007231,-0.017505,0.007790,-0.026162,-0.019518,-0.029942,3.0
2022-04-01,0.048393,0.037186,0.023711,0.034915,0.007754,0.032213,0.037130,3.0


In [52]:
##clarify which market will be using ff or SPX
mark_name = 'SPX'
#mark_name = 'Mkt-RF'

##function to optimize which factor is best for each cycle
def get_fact(model,returns,sort,long_only=True):
    df = []
    #clean df to just hav ethe correct columns
    cycle_df = returns.apply(pd.to_numeric, errors='coerce').dropna()
    #round cycle number if not whole number
    cycle_df[model] = cycle_df[model].round()
    for i in range(1,5):
        des = cycle_df.loc[cycle_df[model]==i].describe().drop([model,mark_name], axis=1)
        if sort == 'Sharpe':
            #get worst and best factor based on regime sharpe ration
            optimal_max = (des.loc['mean']/des.loc['std']).idxmax()
            optimal_min = (des.loc['mean']/des.loc['std']).idxmin()
        elif sort == 'Return':
            #get worst and best factor based on avg return
            optimal_max = des.loc['mean'].idxmax()
            optimal_min = des.loc['mean'].idxmin()
        elif sort == 'VaR':
            #get worst and best factor based on lowest 95% VaR
            optimal_max = cycle_df.loc[cycle_df[model]==i].drop([model,mark_name], axis=1).quantile(0.05).idxmin()
            optimal_min = cycle_df.loc[cycle_df[model]==i].drop([model,mark_name], axis=1).quantile(0.05).idxmax()

        #long only vs long and short
        if long_only:
            optimal = optimal_max
        else:
            optimal = [optimal_max, optimal_min]
        df.append(optimal)
    return df  

In [53]:
#slowly roll over testing from 75% of testing from start to finish
def get_opt_overtime(model,returns,sort):
    df = returns.apply(pd.to_numeric, errors='coerce').dropna()
    df_half = round(len(df)*(0.75))
    in_return = []
    for x in range(df_half):
        in_sample = df[0+x:df_half+x]
        in_return.append(get_fact(model,in_sample,sort))
    opt = pd.DataFrame(in_return).mode().values.tolist()[0]
    return opt

In [54]:
## function that takes in the opt strategy and creates the returns data
def returns_data(model, returns, opt,long_only=True):    
    strat = returns.apply(pd.to_numeric, errors='coerce').dropna()
    strat[model] = strat[model].round()
    #set up empty cells with for the weights
    for col in strat.columns:
        if model in col: 
            continue
        if mark_name in col:
            continue
        else:
            strat[col + '_weight'] = 0
    #weight the cycle returns
    if long_only:
        for col in strat.columns:
            if ('weight' in col) & (opt[0] in col):
                strat[col] = np.where(strat[model] == 1,1,strat[col])
            if ('weight' in col) & (opt[1] in col):
                strat[col] = np.where(strat[model] == 2,1,strat[col])
            if ('weight' in col) & (opt[2] in col):
                strat[col] = np.where(strat[model] == 3,1,strat[col])
            if ('weight' in col) & (opt[3] in col):
                strat[col] = np.where(strat[model] == 4,1,strat[col])
    else:
        #long
        for col in strat.columns:
            if ('weight' in col) & (opt[0][0] in col):
                strat[col] = np.where(strat[model] == 1,1,strat[col])
            if ('weight' in col) & (opt[1][0] in col):
                strat[col] = np.where(strat[model] == 2,1,strat[col])
            if ('weight' in col) & (opt[2][0] in col):
                strat[col] = np.where(strat[model] == 3,1,strat[col])
            if ('weight' in col) & (opt[3][0] in col):
                strat[col] = np.where(strat[model] == 4,1,strat[col])
        #short
        for col in strat.columns:
            if ('weight' in col) & (opt[0][1] in col):
                strat[col] = np.where(strat[model] == 1,-1,strat[col])
            if ('weight' in col) & (opt[1][1] in col):
                strat[col] = np.where(strat[model] == 2,-1,strat[col])
            if ('weight' in col) & (opt[2][1] in col):
                strat[col] = np.where(strat[model] == 3,-1,strat[col])
            if ('weight' in col) & (opt[3][1] in col):
                strat[col] = np.where(strat[model] == 4,-1,strat[col])

    #combine the returns - ETF
    strat['Rotation_return'] = strat['min_var']*strat['min_var_weight'] + \
                                strat['revenue']*strat['revenue_weight'] + \
                                strat['growth']*strat['growth_weight'] + \
                                strat['value']*strat['value_weight'] + \
                                strat['quality']*strat['quality_weight'] + \
                                strat['Momentum']*strat['Momentum_weight']
    ##combine the returns - FF
    # strat['Rotation_return'] = strat['SMB']*strat['SMB_weight'] + \
    #                             strat['HML']*strat['HML_weight'] + \
    #                             strat['RMW']*strat['RMW_weight'] + \
    #                             strat['CMA']*strat['CMA_weight'] + \
    #                             strat['Mom']*strat['Mom_weight']
    strat = strat[[mark_name,'Rotation_return',model]]   
    
    #get cumaltive returns
    strat['Market'] = (strat[mark_name]+1).cumprod()-1
    strat['Strategy'] = (strat['Rotation_return']+1).cumprod()-1
    return strat

In [55]:
## function to print results of test data
def stats(df, time,market=False):
    #get market data
    mkt_vol = df[mark_name].std() * np.sqrt(time)
    mkt_ret = (df['Market'][-1]+1)**(1/(len(df)/time))-1
    mkt_sharpe = mkt_ret/mkt_vol
    #get strategy data
    strat_vol = df['Rotation_return'].std() * np.sqrt(time)
    strat_ret = (df['Strategy'][-1]+1)**(1/(len(df)/time))-1
    strat_sharpe = strat_ret/strat_vol

    #print results
    if market:
        print("Market Return: {}%".format((100*mkt_ret).round(2)))
        print("Market Vol: {}%".format((100*mkt_vol).round(2)))
        print("Market Sharpe: {}".format(mkt_sharpe.round(2)))
        return mkt_ret,mkt_vol,mkt_sharpe
    else:
        print("Strategy Return: {}%".format((100*strat_ret).round(2)))
        print("Strategy Vol: {}%".format((100*strat_vol).round(2)))
        print("Strategy Sharpe: {}".format(strat_sharpe.round(2)))
        return strat_ret, strat_vol, strat_sharpe

In [56]:
#pick what dates you want to run the test and train on
train = yc_returns.loc[yc_returns.index < "2012-12-31"]
test = yc_returns.loc[yc_returns.index >= "2012-12-31"]

In [57]:
#get the model name
model = 'Label'

#run optimization function for each strategy
opt_return = get_fact(model,train,"Return")
opt_sharpe = get_fact(model,train,"Sharpe")
opt_var = get_fact(model,train,"VaR")

#print results of optimization
print("Return Strategy for cycle 1-4: {}".format(opt_return))
print("Sharpe Strategy for cycle 1-4: {}".format(opt_sharpe))
print("Var Strategy for cycle 1-4: {}".format(opt_var))

#print results of strategy on test data
print("Market")
mkt_out =stats(returns_data(model, test,opt_return),12,True)
print("\nBest Return Strategy")
returns_out =stats(returns_data(model, test,opt_return),12)
print("\nBest Return Sharpe")
sharpe_out =stats(returns_data(model, test,opt_sharpe),12)
print("\nBest Return Historical VaR")
var_out =stats(returns_data(model, test,opt_var),12)

Return Strategy for cycle 1-4: ['value', 'value', 'Momentum', 'min_var']
Sharpe Strategy for cycle 1-4: ['min_var', 'min_var', 'Momentum', 'min_var']
Var Strategy for cycle 1-4: ['growth', 'Momentum', 'value', 'value']
Market
Market Return: 13.14%
Market Vol: 14.13%
Market Sharpe: 0.93

Best Return Strategy
Strategy Return: 14.79%
Strategy Vol: 16.16%
Strategy Sharpe: 0.92

Best Return Sharpe
Strategy Return: 10.91%
Strategy Vol: 13.35%
Strategy Sharpe: 0.82

Best Return Historical VaR
Strategy Return: 8.99%
Strategy Vol: 18.91%
Strategy Sharpe: 0.48


In [58]:
##get return data
beta_info = returns_data(model, train,opt_return)
#beta_info1 = returns_data(model, test,opt_return)
##orginize test data
X = beta_info[mark_name]
Y = beta_info['Rotation_return']
X = sm.add_constant(X)
##fit test data
reg = sm.OLS(Y, X).fit()
# ##organize train data
# X1 = beta_info1[mark_name]
# Y1 = beta_info1['Rotation_return']
# X1 = sm.add_constant(X1)
# ##fit train data
# reg1 = sm.OLS(Y1, X1).fit()

alpha_beta = reg.params

print("Alpha and Beta")
print("On Test data")
print(alpha_beta)
#print("")
# print("On Train data")
# print(reg1.params)

Alpha and Beta
On Test data
const    0.011967
SPX      0.904351
dtype: float64


## Long - Short

In [59]:
#get the model name
model = 'Label'
#run funciton to optimize for each strategy
opt_return1 = get_fact(model,train,"Return",False)
opt_sharpe1 = get_fact(model,train,"Sharpe",False)
opt_var1 = get_fact(model,train,"VaR",False)

#print results of the optimization function
print("Return Strategy for cycle 1-4: {}".format(opt_return1))
print("Sharpe Strategy for cycle 1-4: {}".format(opt_sharpe1))
print("Var Strategy for cycle 1-4: {}".format(opt_var1))
## print results of LS opt strat on test data
# print("Market")
# stats(returns_data(model, test,opt_return),12,True)
print("\nBest Return Strategy")
returns_out1 =stats(returns_data(model, test,opt_return1,False),12)
print("\nBest Return Sharpe")
sharpe_out1 =stats(returns_data(model, test,opt_sharpe1,False),12)
print("\nBest Return Historical VaR")
var_out1 =stats(returns_data(model, test,opt_var1,False),12)

Return Strategy for cycle 1-4: [['value', 'min_var'], ['value', 'min_var'], ['Momentum', 'value'], ['min_var', 'value']]
Sharpe Strategy for cycle 1-4: [['min_var', 'Momentum'], ['min_var', 'Momentum'], ['Momentum', 'value'], ['min_var', 'revenue']]
Var Strategy for cycle 1-4: [['growth', 'min_var'], ['Momentum', 'min_var'], ['value', 'quality'], ['value', 'min_var']]

Best Return Strategy
Strategy Return: 5.47%
Strategy Vol: 12.42%
Strategy Sharpe: 0.44

Best Return Sharpe
Strategy Return: -2.36%
Strategy Vol: 8.64%
Strategy Sharpe: -0.27

Best Return Historical VaR
Strategy Return: -0.99%
Strategy Vol: 10.63%
Strategy Sharpe: -0.09


In [60]:
##get the return data
beta_info_ls = returns_data(model, train,opt_return1,False)
#beta_info1_ls = returns_data(model, test,opt_return1,False)
##orginize test data 
X = beta_info_ls[mark_name]
Y = beta_info_ls['Rotation_return']
X = sm.add_constant(X)
##fit test data
reg_ls = sm.OLS(Y, X).fit()
# ##organize train data
# X1 = beta_info1_ls[mark_name]
# Y1 = beta_info1_ls['Rotation_return']
# X1 = sm.add_constant(X1)
# ##train test data
# reg1_ls = sm.OLS(Y1, X1).fit()

alpha_beta_ls = reg.params

print("Alpha and Beta")
print("On Test data")
print(alpha_beta_ls)
# print("")
# print("On Train data")
# print(reg1_ls.params)

Alpha and Beta
On Test data
const    0.011967
SPX      0.904351
dtype: float64


## Looking at model traing over time and not simple train and testing

In [61]:
#get the model name
model = 'Label'

#run optimization function for each strategy
opt_return_ot = get_opt_overtime(model,train,"Return")
opt_sharpe_ot = get_opt_overtime(model,train,"Sharpe")
opt_var_ot = get_opt_overtime(model,train,"VaR")

#print results of optimization
print("Return Strategy for cycle 1-4: {}".format(opt_return_ot))
print("Sharpe Strategy for cycle 1-4: {}".format(opt_sharpe_ot))
print("Var Strategy for cycle 1-4: {}".format(opt_var_ot))

#print results of strategy on test data
print("Market")
mkt_out_ot =stats(returns_data(model, test,opt_return_ot),12,True)
print("\nBest Return Strategy")
returns_out_ot = stats(returns_data(model, test,opt_return_ot),12)
print("\nBest Return Sharpe")
sharpe_out_ot =stats(returns_data(model, test,opt_sharpe_ot),12)
print("\nBest Return Historical VaR")
var_out_ot =stats(returns_data(model, test,opt_var_ot),12)

Return Strategy for cycle 1-4: ['value', 'value', 'Momentum', 'min_var']
Sharpe Strategy for cycle 1-4: ['min_var', 'min_var', 'Momentum', 'min_var']
Var Strategy for cycle 1-4: ['value', 'Momentum', 'value', 'value']
Market
Market Return: 13.14%
Market Vol: 14.13%
Market Sharpe: 0.93

Best Return Strategy
Strategy Return: 14.79%
Strategy Vol: 16.16%
Strategy Sharpe: 0.92

Best Return Sharpe
Strategy Return: 10.91%
Strategy Vol: 13.35%
Strategy Sharpe: 0.82

Best Return Historical VaR
Strategy Return: 7.74%
Strategy Vol: 18.9%
Strategy Sharpe: 0.41


#Long short

In [62]:
#get the model name
model = 'Label'
#run funciton to optimize for each strategy
opt_return_ot1 = get_fact(model,train,"Return",False)
opt_sharpe_ot1 = get_fact(model,train,"Sharpe",False)
opt_var_ot1 = get_fact(model,train,"VaR",False)

#print results of the optimization function
print("Return Strategy for cycle 1-4: {}".format(opt_return_ot1))
print("Sharpe Strategy for cycle 1-4: {}".format(opt_sharpe_ot1))
print("Var Strategy for cycle 1-4: {}".format(opt_var_ot1))
## print results of LS opt strat on test data
# print("Market")
# stats(returns_data(model, test,opt_return),12,True)
print("\nBest Return Strategy")
returns_out_ot1 =stats(returns_data(model, test,opt_return_ot1,False),12)
print("\nBest Return Sharpe")
sharpe_out_ot1 =stats(returns_data(model, test,opt_sharpe_ot1,False),12)
print("\nBest Return Historical VaR")
var_out_ot1 =stats(returns_data(model, test,opt_var_ot1,False),12)

Return Strategy for cycle 1-4: [['value', 'min_var'], ['value', 'min_var'], ['Momentum', 'value'], ['min_var', 'value']]
Sharpe Strategy for cycle 1-4: [['min_var', 'Momentum'], ['min_var', 'Momentum'], ['Momentum', 'value'], ['min_var', 'revenue']]
Var Strategy for cycle 1-4: [['growth', 'min_var'], ['Momentum', 'min_var'], ['value', 'quality'], ['value', 'min_var']]

Best Return Strategy
Strategy Return: 5.47%
Strategy Vol: 12.42%
Strategy Sharpe: 0.44

Best Return Sharpe
Strategy Return: -2.36%
Strategy Vol: 8.64%
Strategy Sharpe: -0.27

Best Return Historical VaR
Strategy Return: -0.99%
Strategy Vol: 10.63%
Strategy Sharpe: -0.09


##output results into an excel file

In [63]:
out_index = ['mkt', 'mkt ret','mkt vol','mkt sharpe',
            'Long Only','strat','strat ret','strat vol',
                'strat sharpe','strat Beta','strat alpha',
            'Long Short','strat','strat ret','strat vol',
                'strat sharpe','strat Beta','strat alpha']

out_data = [mark_name, mkt_out[0],mkt_out[1],mkt_out[2],
            '', opt_return, returns_out[0],returns_out[1],
            returns_out[2], alpha_beta[1],alpha_beta[0],
            '', opt_return1, returns_out1[0],returns_out1[1],
            returns_out1[2], alpha_beta_ls[1],alpha_beta_ls[0]]

In [66]:
out_file = 'high_yield_out.xlsx'
pd.DataFrame(data = out_data, index = out_index).to_excel(out_file)